In [38]:
import torch
from torch import nn
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt
from datetime import datetime

class TimeSeriesDataset(Dataset):
    def __init__(self, dataframe, seq_len=7*24, pred_len=24):
        self.seq_len = seq_len
        self.pred_len = pred_len

        self.dataframe = self._preprocess(dataframe)

    def _preprocess(self, df):
        # If there are any missing values, fill them with the previous value in time-series
        df.fillna(method='ffill', inplace=True)

        # Normalize numerical columns to range [0, 1]
        scaler = MinMaxScaler()
        numerical_cols = df.select_dtypes(include=[np.number]).columns
        df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

        # One-hot encode categorical variables
        categorical_cols = df.select_dtypes(include=['object']).columns
        if not categorical_cols.empty:
            encoder = OneHotEncoder()
            encoded = encoder.fit_transform(df[categorical_cols])
            encoded_df = pd.DataFrame(encoded.toarray(), columns=encoder.get_feature_names(categorical_cols))
            
            # Drop original categorical columns and merge with encoded ones
            df.drop(columns=categorical_cols, inplace=True)
            df = pd.concat([df, encoded_df], axis=1)
        
        return df

    def __len__(self):
        return len(self.dataframe) - self.seq_len - self.pred_len + 1

    def __getitem__(self, idx):
        x = self.dataframe.iloc[idx:idx+self.seq_len]
        y = self.dataframe.iloc[idx+self.seq_len:idx+self.seq_len+self.pred_len, -56:] # Assuming last 56 columns are power values
        return torch.Tensor(x.values), torch.Tensor(y.values).reshape(-1)  # flatten y values
    
# Define LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.n_layers, x.size(0), self.hidden_dim).to(x.device) # Initialize hidden state
        c0 = torch.zeros(self.n_layers, x.size(0), self.hidden_dim).to(x.device) # Initialize cell state

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out


In [11]:
# Load the excel file
df = pd.read_excel('/home/kimyirum/EMS/ict-2023-ems/load/data/merged_data.xlsx')

# Initialize our dataset class
dataset = TimeSeriesDataset(df)

# Define the split sizes for train, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

# Split dataset
train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

In [47]:
########################################## Hyperparameters ##########################################
input_dim = len(train_set[0][0][0]) # 63
output_dim = 24*56
hidden_dim = 256
n_layers = 7
learning_rate = 0.001
num_epochs = 150
batch_size = 64
########################################################################################################

# DataLoader
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_set, batch_size=batch_size, shuffle=False)

# Initialize model, loss and optimizer
model = LSTMModel(input_dim, hidden_dim, output_dim, n_layers)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

########################################## Training loop ##########################################
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validate the model
    model.eval()
    with torch.no_grad():
        val_losses = []
        for data, targets in val_loader:
            outputs = model(data)
            loss = criterion(outputs, targets)
            val_losses.append(loss.item())
        avg_val_loss = sum(val_losses) / len(val_losses)
        print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}')
########################################################################################################

########################################## Evaluate the model ##########################################
# DataLoader for test set
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False)
model.eval()
with torch.no_grad():
    all_targets = []
    all_outputs = []
    for data, targets in test_loader:
        outputs = model(data)
        all_targets.append(targets.numpy())
        all_outputs.append(outputs.numpy())

# Flatten targets and outputs to calculate metrics
all_targets = np.concatenate(all_targets).flatten()
all_outputs = np.concatenate(all_outputs).flatten()

# Calculate MAE, MSE and RMSE
mae = mean_absolute_error(all_targets, all_outputs)
mse = mean_squared_error(all_targets, all_outputs)
rmse = sqrt(mse)

print(f'MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}')
########################################################################################################


Epoch [1/150], Validation Loss: 0.0703
Epoch [2/150], Validation Loss: 0.0501
Epoch [3/150], Validation Loss: 0.0411
Epoch [4/150], Validation Loss: 0.0328
Epoch [5/150], Validation Loss: 0.0261
Epoch [6/150], Validation Loss: 0.0212
Epoch [7/150], Validation Loss: 0.0196
Epoch [8/150], Validation Loss: 0.0182
Epoch [9/150], Validation Loss: 0.0179
Epoch [10/150], Validation Loss: 0.0177
Epoch [11/150], Validation Loss: 0.0157
Epoch [12/150], Validation Loss: 0.0153
Epoch [13/150], Validation Loss: 0.0138
Epoch [14/150], Validation Loss: 0.0148
Epoch [15/150], Validation Loss: 0.0147
Epoch [16/150], Validation Loss: 0.0215
Epoch [17/150], Validation Loss: 0.0154
Epoch [18/150], Validation Loss: 0.0131
Epoch [19/150], Validation Loss: 0.0116
Epoch [20/150], Validation Loss: 0.0105
Epoch [21/150], Validation Loss: 0.0100
Epoch [22/150], Validation Loss: 0.0093
Epoch [23/150], Validation Loss: 0.0079
Epoch [24/150], Validation Loss: 0.0080
Epoch [25/150], Validation Loss: 0.0085
Epoch [26

In [48]:
results_folder = "/home/kimyirum/EMS/ict-2023-ems/load/results"
hyperparams = {
    '\tlearning_rate': learning_rate,
    '\tbatch_size': batch_size,
    '\tnum_epochs': num_epochs,
    '\thidden_dim': hidden_dim,
    '\tn_layers': n_layers
}
metrics = {
    '\tMAE': mae,
    '\tMSE': mse,
    '\tRMSE': rmse,
}

now = datetime.now()
now_str = now.strftime('%Y%m%d_%H%M%S')
filename_metrics = f'{now_str}.txt'
filename_model = f'model_{now_str}.pt'

# Save the model parameters
torch.save(model.state_dict(), results_folder+filename_model)

# Open the file in write mode ('w')
with open(results_folder+filename_metrics, 'w') as f:
    # Write hyperparameters
    f.write('Hyperparameters:\n')
    for key, value in hyperparams.items():
        f.write(f'{key}: {value}\n')
    
    # Write metrics
    f.write('\nMetrics:\n')
    for key, value in metrics.items():
        f.write(f'{key}: {value}\n')

In [49]:
# Assuming df is the original dataset and it includes a 'date' column

# Get the building names
building_names = df.columns[-56:]  # adjust this as necessary

# DataLoader for test set
test_loader = DataLoader(dataset=test_set, batch_size=1, shuffle=False)

# Get the first sequence and its target from the test set
real_sequence, real_target = next(iter(test_loader))

# Switch model to eval mode
model.eval()

# Make prediction
with torch.no_grad():
    prediction = model(real_sequence)

# Remove batch dimension, reshape and convert prediction to numpy array
prediction = prediction.squeeze(0).reshape(24, 56).numpy()

# Reshape real_target and convert it to numpy array
real_target = real_target.view(24, 56).numpy()

# Calculate error (difference between real target and prediction)
error = real_target - prediction

# Create DataFrame for prediction
predicted_df = pd.DataFrame(prediction, columns=building_names)
real_target_df = pd.DataFrame(real_target, columns=building_names)
error_df = pd.DataFrame(error, columns=building_names)

# print("Predicted Values for next 24 hours:")
# print(predicted_df)

# print("Real Values for next 24 hours:")
# print(real_target_df)

print("Error for next 24 hours:")
print(error_df)


Error for next 24 hours:
      0_SV-2    1_SV-5    2_SV-6    3_SV-7  4_HV-NM1  5_HV-NM2   6_고압콘덴샤  \
0  -0.010489 -0.023625  0.000900 -0.028915 -0.002541 -0.014882 -0.006714   
1  -0.016766 -0.023331  0.000096 -0.030150  0.002016 -0.007350  0.002063   
2  -0.020153 -0.022179 -0.002090 -0.026695 -0.004948 -0.024024 -0.011096   
3  -0.028973 -0.026115 -0.002143 -0.013792  0.003501 -0.021451 -0.005518   
4  -0.025696 -0.022664 -0.001272 -0.032025  0.000949 -0.029233 -0.001101   
5  -0.035808 -0.024842 -0.002239 -0.022164 -0.003136 -0.022036 -0.008269   
6  -0.012555 -0.017803  0.002190 -0.020562  0.006935 -0.024767 -0.010656   
7  -0.002873 -0.015469  0.000654 -0.032108  0.001787 -0.033026 -0.009859   
8  -0.020079 -0.014451  0.000765 -0.013958 -0.003318 -0.030697 -0.011499   
9  -0.010661 -0.010832  0.003604 -0.016078  0.002204 -0.011631 -0.003296   
10 -0.020291 -0.018524  0.002049 -0.023115  0.000176 -0.034037 -0.017167   
11 -0.018572 -0.021845 -0.007100 -0.013061  0.001176 -0.025750 